In [1]:
import sys

# change the next line to reflect where you have downloaded the source code
sys.path.insert(0, "/Users/kvolk/Documents/GitHub/SBDynT/src")
import sbdynt as sbd

# **Functions that query JPL services for small bodies and planets**

**Querying the JPL small body database**: querying a small body's orbit from JPL's small body databse returns a flag 
(1 for sucess, 0 for failure), the epoch, the heliocentric cartesian coordinates 
for the best fit plus however many clones is specified, and the weights of the clones (all equally weighted for the default gaussian cloning method, but other cloning schemes will be added in the future)

In [2]:
# example with no clones
small_body = "K14X40T"
flag, epoch, x, y, z, vx, vy, vz, weights = sbd.query_sb_from_jpl(
    des=small_body, clones=0
)
if flag:
    print("queried %s and returned at epoch %f" % (small_body, epoch))
    print(
        "cartesian heliocentric position (au), velocity (au/year):\n %e %e %e\n %e %e %e"
        % (x, y, z, vx, vy, vz)
    )


# example with 5 clones,
# the first index on the returned variables is best fit,
# followed by 5 clones sampled from the covariance matrix
clones = 5
flag, epoch, x, y, z, vx, vy, vz, weights = sbd.query_sb_from_jpl(
    des=small_body, clones=clones
)
if flag:
    print("queried %s and returned at epoch %f" % (small_body, epoch))
    print("cartesian heliocentric position (au), velocity (au/year)")
    print("best-fit orbit:")
    i = 0
    print(6 * "%15.8e " % (x[i], y[i], z[i], vx[i], vy[i], vz[i]))
    print("cloned orbits:")
    for i in range(1, clones):
        print(6 * "%15.8e " % (x[i], y[i], z[i], vx[i], vy[i], vz[i]))

queried K14X40T and returned at epoch 2457293.500000
cartesian heliocentric position (au), velocity (au/year):
 1.993027e+01 2.505351e+01 -5.495645e-01
 -9.122049e-01 7.794144e-01 -1.538141e-01
queried K14X40T and returned at epoch 2457293.500000
cartesian heliocentric position (au), velocity (au/year)
best-fit orbit:
 1.99302698e+01  2.50535068e+01 -5.49564464e-01 -9.12204938e-01  7.79414378e-01 -1.53814146e-01 
cloned orbits:
 1.99302656e+01  2.50534940e+01 -5.49566609e-01 -9.12209993e-01  7.79407166e-01 -1.53811089e-01 
 1.99301333e+01  2.50533405e+01 -5.49554852e-01 -9.12198523e-01  7.79408059e-01 -1.53812259e-01 
 1.99302677e+01  2.50534874e+01 -5.49561683e-01 -9.12209221e-01  7.79405473e-01 -1.53814134e-01 
 1.99300773e+01  2.50532744e+01 -5.49564309e-01 -9.12188146e-01  7.79419565e-01 -1.53813954e-01 


**If you want just a best-fit orbit at a specific epoch for one or more 
small bodies, you can query JPL Horizons instead** (it is currently not possible
to add clones at a specific epoch or for multiple objects at once due to the
fact that the covariance matrix for each object is at a non-use-determined
eopch that differs for each small body. We plan to add this capability at a 
later date by integrating bodies to a common epoch after cloning)


Note that this will not exactly match the position above because often
Horizons uses a different orbit-fit than the small body database

In [3]:
# single object example
sbody = "K14X40T"
epoch = 2457019.5
(flag, xbf, ybf, zbf, vxbf, vybf, vzbf) = sbd.query_sb_from_horizons(
    des=sbody, epoch=epoch
)
if flag:
    print("queried %s and returned at epoch %f" % (sbody, epoch))
    print("cartesian heliocentric position (au), velocity (au/year)")
    print(3 * "%15.8e " % (xbf, ybf, zbf))
    print(3 * "%15.8e " % (vxbf, vybf, vzbf))
print("\n")

queried K14X40T and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
 2.06080559e+01  2.44602462e+01 -4.34010782e-01 
-8.94696420e-01  8.02276834e-01 -1.54242125e-01 




In [4]:
# multiple objects example
# note that designations can be packed, unpacked, numbers, comets, etc
list_of_sbodies = ["K14X40T", "2016 SW50", "15760", "29P", "179P/Jedicke"]
epoch = 2457019.5
ntp = len(list_of_sbodies)
(flag, x, y, z, vx, vy, vz) = sbd.query_sb_from_horizons(
    des=list_of_sbodies, epoch=epoch
)
if flag:
    for n in range(0, ntp):
        print()
        print(
            "queried %s and returned at epoch %f" % (list_of_sbodies[n], epoch)
        )
        print("cartesian heliocentric position (au), velocity (au/year)")
        print(3 * "%15.8e " % (x[n], y[n], z[n]))
        print(3 * "%15.8e " % (vx[n], vy[n], vz[n]))


queried K14X40T and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
 2.06080559e+01  2.44602462e+01 -4.34010782e-01 
-8.94696420e-01  8.02276834e-01 -1.54242125e-01 

queried 2016 SW50 and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
 3.03507298e+01  1.54114154e+01 -1.21830433e+01 
-4.30637890e-01  1.06543484e+00  5.67813014e-01 

queried 15760 and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
 3.51679007e+01  2.14900270e+01  8.33927151e-01 
-5.00071912e-01  8.74610680e-01  3.32892176e-02 

queried 29P and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
-1.68501340e+00 -5.78274844e+00 -8.49470774e-01 
 2.43990351e+00 -6.37653588e-01  2.26432820e-01 

queried 179P/Jedicke and returned at epoch 2457019.500000
cartesian heliocentric position (au), velocity (au/year)
-4.64752344e+00 -5.72543094e+00  2.40730903e+00 
 1.

**You can also query the planet properties and positions for the epoch 
returned by the small body's orbit query (showing just Jupiter as an example)**

In [5]:
planet = "jupiter"  # not case sensitive
epoch = 2457019.5
(flag, mass, radius, [plx, ply, plz], [plvx, plvy, plvz]) = (
    sbd.query_horizons_planets(obj=planet, epoch=epoch)
)
if flag:
    print("queried %s and returned at epoch %f" % (planet, epoch))
    print("mass (solar masses) %e and radius (au) %e" % (mass, radius))
    print("cartesian heliocentric position (au), velocity (au/year)")
    print(3 * "%15.8e " % (plx, ply, plz))
    print(3 * "%15.8e " % (plvx, plvy, plvz))

queried jupiter and returned at epoch 2457019.500000
mass (solar masses) 9.547919e-04 and radius (au) 4.778945e-04
cartesian heliocentric position (au), velocity (au/year)
-3.70760331e+00  3.81380893e+00  6.71243900e-02 
-2.01106301e+00 -1.79277124e+00  5.24449611e-02 
